<a href="https://colab.research.google.com/github/achanhon/coursdeeplearningcolab/blob/master/TP_ADVERSAIRE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TP deep learning sous attaque adversaire (2024)
####Adrien Chan-Hon-Tong
####TP réalisé à partir de résultats de Pol Labarbarie


L'objet de ce TP est de démontrer
- la faciliter de produire des attaques adversaires "white box" sur des réseaux naifs quelles soient invisibles ou par patch
- mais que cela est beaucoup plus dur sur un réseau robustifier (cas invisible)
- ou encore qu'il est beaucoup plus difficile de produire des attaques "transferable"

## generalité
Commençons par télécharger 10 images d'imagenet.

In [1]:
!rm -f *
!wget https://httpmail.onera.fr/21/500df56b6a7a034ecc2f3e0345f1ce9cYqsE3G/data.zip
!unzip data.zip
!ls

rm: cannot remove 'build': Is a directory
rm: cannot remove 'sample_data': Is a directory
--2024-10-05 09:40:31--  https://httpmail.onera.fr/21/500df56b6a7a034ecc2f3e0345f1ce9cYqsE3G/data.zip
Resolving httpmail.onera.fr (httpmail.onera.fr)... 144.204.16.9
Connecting to httpmail.onera.fr (httpmail.onera.fr)|144.204.16.9|:443... failed: No route to host.
unzip:  cannot find or open data.zip, data.zip.zip or data.zip.ZIP.
build  sample_data


Affichons les : les 5 premières sont des "avions" et les 5 suivantes des "requins"

In [2]:
import torch
import torchvision
import matplotlib.pyplot as plt

x = [torchvision.io.read_image(str(i)+".png") for i in range(10)]
x = torch.stack(x,dim=0).float()/255

visu = torchvision.utils.make_grid(x, nrow=5)
plt.imshow(visu.permute(1, 2, 0).numpy())
plt.show()

RuntimeError: [Errno 2] No such file or directory: '0.png'

In [ ]:
SHARK, PLANE = [2, 3, 4], [403, 404, 405]
normalize = torchvision.transforms.Normalize(
    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
)
resnet = torchvision.models.resnet101(
    weights=torchvision.models.ResNet101_Weights.IMAGENET1K_V1
).eval()

with torch.no_grad():
    z = resnet(normalize(x))
    _,z = z.max(1)
    print(z)

On voit que le réseau classe correctement ces images.

## Attaque standard "white box"

On va maintenant rajouter à ces images un petit bruit "invisible" pour l'oeil mais perturbant pour le réseau.

In [ ]:
y = torch.Tensor([403, 405, 404, 405, 404,   4,   3,   3,   3,   2]).long()
cefunction = torch.nn.CrossEntropyLoss()
attaque = torch.nn.Parameter(torch.zeros(x.shape))
optimizer = torch.optim.SGD([attaque],lr=0.005)
for i in range(10):
  z = resnet(normalize(x+attaque))
  ce = cefunction(z,y)
  print(i,float(ce))
  ce = -ce # on veut MAXIMISER la cross entropy puisqu'on attaque
  optimizer.zero_grad()
  ce.backward()
  attaque.grad = attaque.grad.sign()
  optimizer.step()
  with torch.no_grad():
      # l'attaque doit être invisible
      attaque = torch.clamp(attaque, -10./255,+10./255)

      # attaque+x doit être entre 0 et 1
      lowbound = -x
      uppbound = 1-x
      attaque = lowbound*(attaque<lowbound).float() + uppbound*(attaque>uppbound).float() + attaque *(attaque>=lowbound).float()*(attaque<=uppbound).float()

  attaque = torch.nn.Parameter(attaque.clone())
  optimizer = torch.optim.SGD([attaque],lr=0.005)

80% des images "x+attaque" sont désormais mal classées ! (et le label de toutes à changer)
Pourtant, l'attaque ne se voit pas :

In [ ]:
with torch.no_grad():
    z = resnet(normalize(x))
    _,z = z.max(1)
    print(z)
    z = resnet(normalize(x+attaque))
    _,z = z.max(1)
    print(z)

visu = torch.cat([x,x+attaque],dim=0)
visu = torchvision.utils.make_grid(visu, nrow=5)
plt.imshow(visu.permute(1, 2, 0).numpy())
plt.show()

Comment est ce que c'est possible ? Les réseaux ne sont pas du tout lipschitziens...

In [ ]:
with torch.no_grad():
    resnet = torchvision.models.resnet101(
        weights=torchvision.models.ResNet101_Weights.IMAGENET1K_V1
    ).eval()
    resnet.fc = torch.nn.Identity()
    z = resnet(x)
    print(((z[0]-z[5])**2).sum())
    z_ = resnet(x+attaque)
    print(((z[0]-z_[0])**2).sum())

On voit que la représentation de l'image 0 devient presque aussi lointaine à cause de l'attaque que la distance avec l'image 5 !
Alors que nous ne voyons même pas la différence !

____________________________________________________________________________
=> retour aux slides (on revient ici après).
____________________________________________________________________________

# Attaque standard par patch "white box"
Maintenant on va regarder la création d'un patch adversarial : pour rappel, le problème des bruits invisibles c'est l'impossibilité de les faire dans le monde physique et l'existance de défense -> deux choses que les patches peuvent bypasser.

On va mettre un patch 36x36 en haut à gauche (remarquons que si le patch est juste noir, ça change rien).

In [ ]:
mask = torch.zeros(1,3,224,224)
mask[:,:,0:36,0:36] = 1

resnet = torchvision.models.resnet101(
    weights=torchvision.models.ResNet101_Weights.IMAGENET1K_V1
).eval()
with torch.no_grad():
    z = resnet(normalize(x))
    _,z = z.max(1)
    print(z)
    z = resnet(normalize(x*(1-mask)))
    _,z = z.max(1)
    print(z)

visu = torch.cat([x,x*(1-mask)],dim=0)
visu = torchvision.utils.make_grid(visu, nrow=5)
plt.imshow(visu.permute(1, 2, 0).numpy())
plt.show()

mais s'il est optimisé ?

In [ ]:
y = torch.Tensor([403, 405, 404, 405, 404,   4,   3,   3,   3,   2]).long()
cefunction = torch.nn.CrossEntropyLoss()
attaque = torch.nn.Parameter(torch.rand(1,3,224,224))
optimizer = torch.optim.SGD([attaque],lr=0.1)
for i in range(40):
  z = resnet(normalize(x*(1-mask)+mask*attaque))
  ce = cefunction(z,y)
  print(i,float(ce))
  ce = -ce # on veut MAXIMISER la cross entropy puisqu'on attaque
  optimizer.zero_grad()
  ce.backward()
  attaque.grad = attaque.grad.sign()
  optimizer.step()
  with torch.no_grad():
      # l'attaque doit être dans le domaine image
      attaque = torch.clamp(attaque, 0,1)

  attaque = torch.nn.Parameter(attaque.clone())
  if i<20:
      optimizer = torch.optim.SGD([attaque],lr=0.1)
  else:
      optimizer = torch.optim.SGD([attaque],lr=0.05)

with torch.no_grad():
    z = resnet(normalize(x))
    _,z = z.max(1)
    print(z)
    z = resnet(normalize(x*(1-mask) + attaque*mask))
    _,z = z.max(1)
    print(z)

visu = torch.cat([x,x*(1-mask)+attaque*mask],dim=0)
visu = torchvision.utils.make_grid(visu, nrow=5)
plt.imshow(visu.permute(1, 2, 0).numpy())
plt.show()

Des images ont vu leur label changé (c'est ici pas 100% mais c'est le MÊME patch pour toutes les images - et il n'est pas vraiment optimisé suffisamment longtemps).

## Limites

Ici on montre la facilité de faire une attaque **numérique** contre un réseau **naif** et **connu**.
Heureusement, la situation est très différente contre un réseau *défendu* ou *inconnu* ou quand l'attaque doit être *physiquement réalisable*.

### Réseaux défendus

On trouve très peu de réseaux défendus sur internet pour Imagenet (on trouve surtout des réseaux CIFAR et les rares qu'on peut trouver pour Imagenet comme dans le github https://github.com/MadryLab/robustness sont des réseaux custom).
Aussi, nous allons abandonnés nos avions/requins et faire des petites expériences sur CIFAR10.

Commençons par apprendre un réseau sur CIFAR:

In [1]:
import torch
import torchvision

normalize = torchvision.transforms.Normalize(
    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
)
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),normalize])
trainset = torchvision.datasets.CIFAR10(
    root="build",
    train=True,
    download=True,
    transform=transform,
)

100%|██████████| 170498071/170498071 [00:03<00:00, 54672409.02it/s]


Extracting build/cifar-10-python.tar.gz to build


In [2]:
resnet = torchvision.models.resnet18(
    weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1
).eval()
resnet.fc = torch.nn.Linear(512,10)

trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=64, shuffle=True, num_workers=2
)
cefunction = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet.parameters(), lr=0.0001)
meanloss = torch.zeros(50)
for i,(x,y) in enumerate(trainloader):
    z=resnet(x)
    loss = cefunction(z,y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    with torch.no_grad():
        meanloss[i%50]=loss.clone()
        if i%50==49:
          print(float(meanloss.mean()))
    if i==1000:
        break

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 134MB/s]


1.5134620666503906
1.0475107431411743
0.885348916053772
0.8793100714683533
0.7926767468452454
0.7795549035072327
0.7765563130378723
0.7171671986579895
0.6997859477996826
0.7229669690132141
0.6560990214347839
0.6875853538513184
0.6767550110816956
0.682990312576294
0.6401225924491882


vérifions que la perfo est pas trop mauvaise même si là on a appris vraiment très très très peu :

In [3]:
testset = torchvision.datasets.CIFAR10(
    root="build",
    train=False,
    download=True,
    transform=transform,
)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=500, shuffle=True, num_workers=2
)

with torch.no_grad():
    for x,y in testloader:
        z = resnet(x)
        _,z = z.max(1)
        good = (z==y).float().sum()
        print(float(good/5))
        break

Files already downloaded and verified
79.0


c'est pas "terrible" mais ça ira pour la preuve de concept...

attaquons 10 images bien classée !

In [4]:
I = [i for i in range(500) if y[i]==z[i]]
I = I[0:10]
y,x=y[I],x[I]
#ces 10 là sont bien classées !

cefunction = torch.nn.CrossEntropyLoss()
attaque = torch.nn.Parameter(torch.zeros(x.shape))
optimizer = torch.optim.SGD([attaque],lr=0.005)
for i in range(25):
  z = resnet(x+attaque)
  ce = cefunction(z,y)
  print(i,float(ce))
  ce = -ce # on veut MAXIMISER la cross entropy puisqu'on attaque
  optimizer.zero_grad()
  ce.backward()
  attaque.grad = attaque.grad.sign()
  optimizer.step()
  with torch.no_grad():
    # l'attaque doit être invisible
    attaque = torch.clamp(attaque, -10./255,+10./255)

    # attaque+x doit être entre 0 et 1
    lowbound = -x
    uppbound = 1-x
    attaque = lowbound*(attaque<lowbound).float() + uppbound*(attaque>uppbound).float() + attaque *(attaque>=lowbound).float()*(attaque<=uppbound).float()

  attaque = torch.nn.Parameter(attaque.clone())
  optimizer = torch.optim.SGD([attaque],lr=0.005)

with torch.no_grad():
  z = resnet(x)
  _,z = z.max(1)
  print(z)
  z = resnet(x+attaque)
  _,z = z.max(1)
  print(z)

visu = torch.cat([x,x+attaque],dim=0)
visu = torchvision.utils.make_grid(visu, nrow=5)
plt.imshow(visu.permute(1, 2, 0).numpy())
plt.show()

0 0.11965018510818481
1 2.4779820442199707
2 2.6715633869171143
3 2.875704288482666
4 3.084207773208618
5 3.2871639728546143
6 3.4851067066192627
7 3.678542375564575
8 3.8522675037384033
9 3.9444000720977783
10 4.02327823638916
11 4.048834323883057
12 4.071587562561035
13 4.088166236877441
14 4.101597785949707
15 4.111011505126953
16 4.1196818351745605
17 4.126502513885498
18 4.132266044616699
19 4.136849880218506
20 4.140960693359375
21 4.144164085388184
22 4.148486137390137
23 4.14995002746582
24 4.152886390686035
tensor([1, 8, 6, 4, 8, 0, 3, 7, 2, 1])
tensor([2, 0, 2, 2, 0, 0, 2, 0, 2, 0])


NameError: name 'plt' is not defined

Bon même si on est pas à 10/10, on voit que le modèle (pourtant non convergé) est quand même très sensible...

maintenant regardons si on apprend un modèle **robuste**.

In [5]:
torch.save(resnet,"resnet.pth")
resnetrobuste = torch.load("resnet.pth") #force unrelated copy

trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=64, shuffle=True, num_workers=2
)
cefunction = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnetrobuste.parameters(), lr=0.0001)
meanloss = torch.zeros(50)
for i,(x,y) in enumerate(trainloader):
    #attack x, then update the weight to deal with the fact that z has been attacked
    attaque = torch.nn.Parameter(torch.zeros(x.shape))
    attackoptimizer = torch.optim.SGD([attaque],lr=0.001)
    for _ in range(10):
      z = resnet(x+attaque)
      ce = cefunction(z,y)
      ce = -ce # on veut MAXIMISER la cross entropy puisqu'on attaque
      optimizer.zero_grad()
      ce.backward()
      attaque.grad = attaque.grad.sign()
      optimizer.step()

    #now attaque is frozen
    with torch.no_grad():
        attaque = torch.Tensor(attaque.clone())

    z = resnetrobuste(x+attaque)
    loss = cefunction(z,y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    with torch.no_grad():
        meanloss[i%50]=loss.clone()
        if i%50==49:
          print(float(meanloss.mean()))

        if i%5==4:
          torch.save(resnetrobuste,"tmp.pth")
          resnet = torch.load("tmp.pth") #update the network from which attack is crafted
    if i==400:
        break

<ipython-input-5-cad2277aec7e>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  resnetrobuste = torch.load("resnet.pth") #force unrelated copy
<ipython-input-5-cad2277aec7e>

0.5893574357032776
0.5478807091712952
0.541315495967865
0.5166321396827698
0.5541200041770935
0.5064505934715271
0.5226083993911743
0.5241930484771729


la performance sur les images "normales" devraient avoir baissée

In [6]:
with torch.no_grad():
    for x,y in testloader:
        z = resnetrobuste(x)
        _,z = z.max(1)
        good = (z==y).float().sum()
        print(float(good/5))
        break

70.80000305175781


mais la performance devrait rester similaire sur des images attaquées

In [7]:
I = [i for i in range(500) if y[i]==z[i]]
I = I[0:10]
y,x=y[I],x[I]
#ces 10 là sont bien classées !

cefunction = torch.nn.CrossEntropyLoss()
attaque = torch.nn.Parameter(torch.zeros(x.shape))
optimizer = torch.optim.SGD([attaque],lr=0.005)
for i in range(25):
  z = resnetrobuste(x+attaque)
  ce = cefunction(z,y)
  print(i,float(ce))
  ce = -ce # on veut MAXIMISER la cross entropy puisqu'on attaque
  optimizer.zero_grad()
  ce.backward()
  attaque.grad = attaque.grad.sign()
  optimizer.step()
  with torch.no_grad():
    # l'attaque doit être invisible
    attaque = torch.clamp(attaque, -10./255,+10./255)

    # attaque+x doit être entre 0 et 1
    lowbound = -x
    uppbound = 1-x
    attaque = lowbound*(attaque<lowbound).float() + uppbound*(attaque>uppbound).float() + attaque *(attaque>=lowbound).float()*(attaque<=uppbound).float()

  attaque = torch.nn.Parameter(attaque.clone())
  optimizer = torch.optim.SGD([attaque],lr=0.005)

with torch.no_grad():
  z = resnetrobuste(x)
  _,z = z.max(1)
  print(z)
  z = resnetrobuste(x+attaque)
  _,z = z.max(1)
  print(z)

visu = torch.cat([x,x+attaque],dim=0)
visu = torchvision.utils.make_grid(visu, nrow=5)
plt.imshow(visu.permute(1, 2, 0).numpy())
plt.show()

0 0.1014028787612915
1 1.3002005815505981
2 1.4649325609207153
3 1.6515552997589111
4 1.8567759990692139
5 2.0755035877227783
6 2.3042683601379395
7 2.5269718170166016
8 2.735957622528076
9 2.8491945266723633
10 2.9443023204803467
11 2.9743831157684326
12 3.004638195037842
13 3.024829387664795
14 3.0413613319396973
15 3.055093288421631
16 3.0629239082336426
17 3.075079917907715
18 3.08308744430542
19 3.0888679027557373
20 3.0954604148864746
21 3.100970983505249
22 3.105414390563965
23 3.1106009483337402
24 3.112619400024414
tensor([3, 6, 8, 3, 9, 1, 6, 6, 9, 1])
tensor([3, 6, 4, 3, 0, 6, 0, 6, 3, 3])


NameError: name 'plt' is not defined